# Tutorial: Evaluating Policies

This is a hands-on tutorial to get familiar with the following topics:

 - OpenAI ```gym``` "videogame" environments
 - State-of-the-art methods for RL over continuous control spaces
 - Measuring the performance of a given policy and reporting it

```
$ python -m ars --env_name gym:BipedalWalker-v2 --n_directions 240 --deltas_used 240 --step_size 0.02 \ 
--delta_std 0.0075 --n_workers 6 --n_iter 1000 --address 10.100.228.201:6379
```

```
$ python3 -m ars.run_policy data/lin_policy_plus.npz BipedalWalker-v2 --render --num_rollouts 20
```